### 1.Cite 5 diferenças entre o AdaBooste o GBM.

1. **Tipo de Função de Perda**:
   - **AdaBoost**: Minimiza a **exponential loss** (função de perda exponencial), que penaliza fortemente as previsões incorretas.
   - **GBM**: Permite a escolha de diferentes funções de perda, como **log_loss** para classificação ou **mean_squared_error** para regressão, oferecendo maior flexibilidade.

2. **Peso das Amostras**:
   - **AdaBoost**: Ajusta os pesos das amostras em cada iteração, aumentando o peso das amostras que foram classificadas incorretamente e diminuindo o peso das que foram classificadas corretamente.
   - **GBM**: Ajusta os erros residuais em cada iteração, focando nas amostras com maiores erros residuais da iteração anterior para melhorar a precisão do modelo.

3. **Método de Atualização do Modelo**:
   - **AdaBoost**: Combina os modelos fracos (geralmente árvores de decisão) com base em uma ponderação que depende da precisão do modelo em cada iteração.
   - **GBM**: Constrói novos modelos fracos que tentam corrigir os erros residuais do modelo anterior, acumulando gradualmente melhorias no modelo final.

4. **Robustez ao Overfitting**:
   - **AdaBoost**: Pode ser mais sensível ao overfitting, especialmente quando se usa um grande número de iterações e as amostras são ruidosas ou difíceis de classificar.
   - **GBM**: Geralmente mais robusto ao overfitting, especialmente com o uso de técnicas como `subsample` e `learning_rate` para regularização.

5. **Complexidade e Flexibilidade**:
   - **AdaBoost**: Mais simples e fácil de implementar, mas com menos opções de personalização e ajuste fino.
   - **GBM**: Mais complexo, mas oferece maior flexibilidade com muitos hiperparâmetros ajustáveis, como a taxa de aprendizado (`learning_rate`), o número de árvores (`n_estimators`), e a fração de amostras (`subsample`). 

### 2.Acesse o link Scikit-learn–GBM, leia a explicação (traduza se for preciso) e crie um jupyternotebook contendo o exemplo de classificação e de regressão do GBM.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_hastie_10_2
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split

####  Classificação

In [2]:
X, y = make_hastie_10_2(random_state=0)
X_train, X_test = X[:2000], X[2000:]
y_train, y_test = y[:2000], y[2000:]

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

0.913

#### Regressão

In [3]:
X, y = make_regression(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0)
reg = GradientBoostingRegressor(random_state=0)
reg.fit(X_train, y_train)
reg.predict(X_test[1:2])
reg.score(X_test, y_test)

0.43848663277068134

### 3.Cite 5 Hyperparametrosimportantes no GBM.

1. **loss**: Define a função de perda a ser otimizada. `log_loss` é usada para saídas probabilísticas e é adequada para classificação, enquanto `exponential` usa o algoritmo AdaBoost.

2. **learning_rate**: Controla o quanto cada árvore contribui para o modelo final. Uma taxa de aprendizado menor requer mais árvores (`n_estimators`) para alcançar uma boa performance, mas pode melhorar a generalização.

3. **n_estimators**: Especifica o número de árvores (estágios de boosting) no modelo. Um número maior pode aumentar a performance, mas também aumenta o tempo de treinamento.

4. **subsample**: Define a proporção das amostras utilizadas para treinar cada árvore. Usar um valor menor que 1.0 cria um modelo de boosting estocástico, reduzindo a variância e aumentando o viés.

5. **criterion**: Determina como a qualidade das divisões das árvores é medida. O valor padrão `friedman_mse` é geralmente preferido, pois pode melhorar a performance em certos casos. 

### 4.(Opcional) Utilize o GridSearch para encontrar os melhores hyperparametrospara o conjunto de dados do exemplo

In [4]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1.0],
    'max_depth': [1, 3, 5]
}

grid_search = GridSearchCV(GradientBoostingRegressor(random_state=0),
                           param_grid,
                           cv=3,
                           n_jobs=-1,
                           scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)

print(f"Melhores hiperparâmetros: {grid_search.best_params_}")

best_reg = grid_search.best_estimator_
best_score = grid_search.best_score_

Melhores hiperparâmetros: {'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 200}


### 5.Acessando o artigo do Jerome Friedman (Stochastic) e pensando no nome dado ao StochasticGBM, qual é a maior diferença entre os dois algoritmos?

A maior diferença entre o **Gradient Boosting Machine (GBM)** tradicional e o **Stochastic Gradient Boosting Machine (Stochastic GBM)**, conforme descrito no artigo de Jerome Friedman, reside no uso de amostragem aleatória dos dados para treinar cada árvore de decisão no conjunto de árvores.

### GBM Tradicional:
- **Amostragem Completa**: No GBM tradicional, cada árvore de decisão é treinada usando **todo o conjunto de dados** disponível. Isso significa que em cada iteração do algoritmo, todos os dados de treinamento são utilizados para ajustar a árvore que minimiza o erro residual.
  
### Stochastic GBM:
- **Amostragem Aleatória Parcial**: No Stochastic GBM, apenas uma **subamostra aleatória** dos dados é usada para treinar cada árvore. Essa subamostra é geralmente uma fração dos dados originais, definida pelo hiperparâmetro `subsample`. Por exemplo, se `subsample=0.8`, apenas 80% dos dados são usados para treinar cada árvore em uma iteração específica. 

### Consequências da Diferença:

1. **Redução da Variância**: O uso de uma subamostra em cada iteração do Stochastic GBM introduz uma forma de regularização, que tende a reduzir a variância do modelo. Isso pode ajudar a evitar overfitting, especialmente em conjuntos de dados grandes ou ruidosos.

2. **Aumento da Robustez**: Como cada árvore é treinada em uma amostra diferente dos dados, o modelo final (que é a média ponderada de todas as árvores) tende a ser mais robusto a outliers e a variações nos dados de treinamento.

3. **Menor Tempo de Treinamento**: Treinar cada árvore em uma subamostra menor dos dados pode reduzir o tempo de treinamento em comparação ao GBM tradicional, que utiliza todo o conjunto de dados para cada árvore.

### Resumo:
A maior diferença entre o GBM tradicional e o Stochastic GBM é que o Stochastic GBM introduz aleatoriedade no processo de treinamento, usando apenas uma subamostra dos dados para treinar cada árvore, enquanto o GBM tradicional utiliza o conjunto completo de dados. Essa diferença pode levar a modelos mais robustos e menos propensos ao overfitting no caso do Stochastic GBM.